In [1]:
import pandas as pd
import urllib.request
import datetime as dt
import requests
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
from haversine import haversine, Unit
from shapely.geometry import Point
import geopandas as gpd
from geopandas import GeoDataFrame
import pytz
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, precision_score

In [2]:
# import necessary libraries
import pandas as pd
import os
import glob
  
  
# use glob to get all the csv files 
# in the folder
path = os.getcwd()
# csv_files = glob.glob(os.path.join(path, "temp/*_hard_voting.csv"))
csv_files = glob.glob(os.path.join(path, "*_all_hard_voting.csv"))
  
master_df = pd.DataFrame()
    
# loop over the list of csv files
for f in csv_files:
      
    # read the csv file
    df = pd.read_csv(f)
      
    # print the location and filename
#     print('Location:', f)
#     print('File Name:', f.split("\\")[-1])
      
#     # print the content
#     print('Content:')
#     display(df)
#     print()
    master_df = pd.concat([master_df, df])
master_df = master_df.reset_index()
master_df = master_df.drop(columns=["index", "Unnamed: 0"])

In [3]:
master_df

,timestamp,image_gt,image_pred,goes16_pred,goes17_pred,final_pred,type
0,2020-07-09 19:14:00+00:00,0.0,0.0,0.0,0.0,0.0,test
1,2020-07-09 19:15:00+00:00,0.0,0.0,0.0,0.0,0.0,test
2,2020-07-09 19:16:00+00:00,0.0,0.0,0.0,0.0,0.0,test
3,2020-07-09 19:17:00+00:00,0.0,0.0,0.0,0.0,0.0,test
4,2020-07-09 19:18:00+00:00,0.0,0.0,0.0,0.0,0.0,test
...,...,...,...,...,...,...,...
10327,2020-12-16 22:11:00+00:00,1.0,1.0,0.0,0.0,0.0,test
10328,2020-12-16 22:12:00+00:00,1.0,1.0,0.0,0.0,0.0,test
10329,2020-12-16 22:13:00+00:00,1.0,1.0,0.0,0.0,0.0,test
10330,2020-12-16 22:14:00+00:00,1.0,1.0,0.0,0.0,0.0,test


In [4]:
master_copy_df = master_df[master_df["type"] == "test"].copy()
# print(master_df.shape)
# master_df = master_df[(master_df["timestamp"] >= "2021-02-01 00:00:00+00:00") | (master_df["timestamp"] < "2021-01-01 00:00:00+00:00")]
print(master_copy_df.shape)
master_copy_df.sort_values(by="timestamp").to_csv("master_df.csv")

(5261, 7)


In [5]:
master_copy_df["timestamp"].unique

<bound method Series.unique of 0        2020-07-09 19:14:00+00:00
1        2020-07-09 19:15:00+00:00
2        2020-07-09 19:16:00+00:00
3        2020-07-09 19:17:00+00:00
4        2020-07-09 19:18:00+00:00
                   ...            
10327    2020-12-16 22:11:00+00:00
10328    2020-12-16 22:12:00+00:00
10329    2020-12-16 22:13:00+00:00
10330    2020-12-16 22:14:00+00:00
10331    2020-12-16 22:15:00+00:00
Name: timestamp, Length: 5261, dtype: object>

In [6]:
master_copy_df.shape

(5261, 7)

In [7]:
image_labels = master_copy_df[~master_copy_df["image_gt"].isna()]["image_gt"]
smokeynet_preds = master_copy_df[~master_copy_df["image_gt"].isna()]["image_pred"]
goes16_preds = master_copy_df[~master_copy_df["image_gt"].isna()]["goes16_pred"]
goes17_preds = master_copy_df[~master_copy_df["image_gt"].isna()]["goes17_pred"]
ensemble_preds = master_copy_df[~master_copy_df["image_gt"].isna()]["final_pred"]

smokeynet_baseline_score = accuracy_score(image_labels, smokeynet_preds)
goes16_baseline_score = accuracy_score(image_labels, goes16_preds)
goes17_baseline_score = accuracy_score(image_labels, goes17_preds)
ensemble_score = accuracy_score(image_labels, ensemble_preds)

print("Baseline score SmokeyNet:", smokeynet_baseline_score)
print("Baseline score GOES-16:", goes16_baseline_score)
print("Baseline score GOES-17:", goes17_baseline_score)
print("Ensemble score:", ensemble_score)

Baseline score SmokeyNet: 0.7880631058734081
Baseline score GOES-16: 0.5172020528416651
Baseline score GOES-17: 0.5708040296521574
Ensemble score: 0.568142938604828


In [8]:
#smokeynet baseline
precision_recall_fscore_support(image_labels, smokeynet_preds, average = 'binary')

(0.8563710040522288, 0.7049666419570052, 0.7733279121772717, None)

In [9]:
#ensemble 
precision_recall_fscore_support(image_labels, ensemble_preds, average = 'binary')

(0.9976635514018691, 0.15826538176426982, 0.27319257837492006, None)

In [10]:
#goes16
precision_recall_fscore_support(image_labels, goes16_preds, average = 'binary')

(0.99375, 0.05893254262416605, 0.11126662001399579, None)

In [11]:
#goes17
precision_recall_fscore_support(image_labels, goes17_preds, average = 'binary')

(0.9867256637168141, 0.1653076352853966, 0.2831746031746032, None)

In [12]:
master_copy_df["weighted_avg"]  = ((0.7907061836809367 * smokeynet_preds) + (0.5181119648737651 * goes16_preds) + (0.5781192828393706 * goes17_preds)) / (0.7907061836809367 + 0.5181119648737651 + 0.5781192828393706)

master_copy_df["thresh2"] = master_copy_df["weighted_avg"] >= 0.2
master_copy_df["thresh2"] = master_copy_df["thresh2"].astype(int)

#ideally we would need > ~0.275  and <= ~0.306 for the threshold
#we can listen to either SmokeyNet and GOES17
master_copy_df["thresh3"] = master_copy_df["weighted_avg"] >= 0.3
master_copy_df["thresh3"] = master_copy_df["thresh3"].astype(int)

master_copy_df["thresh4"] = master_copy_df["weighted_avg"] >= 0.4
master_copy_df["thresh4"] = master_copy_df["thresh4"].astype(int)

master_copy_df["thresh5"] = master_copy_df["weighted_avg"] >= 0.5
master_copy_df["thresh5"] = master_copy_df["thresh5"].astype(int)

master_copy_df["thresh6"] = master_copy_df["weighted_avg"] >= 0.6
master_copy_df["thresh6"] = master_copy_df["thresh6"].astype(int)

master_copy_df["thresh7"] = master_copy_df["weighted_avg"] >= 0.7
master_copy_df["thresh7"] = master_copy_df["thresh7"].astype(int)

master_copy_df


,timestamp,image_gt,image_pred,goes16_pred,goes17_pred,final_pred,type,weighted_avg,thresh2,thresh3,thresh4,thresh5,thresh6,thresh7
0,2020-07-09 19:14:00+00:00,0.0,0.0,0.0,0.0,0.0,test,0.000000,0,0,0,0,0,0
1,2020-07-09 19:15:00+00:00,0.0,0.0,0.0,0.0,0.0,test,0.000000,0,0,0,0,0,0
2,2020-07-09 19:16:00+00:00,0.0,0.0,0.0,0.0,0.0,test,0.000000,0,0,0,0,0,0
3,2020-07-09 19:17:00+00:00,0.0,0.0,0.0,0.0,0.0,test,0.000000,0,0,0,0,0,0
4,2020-07-09 19:18:00+00:00,0.0,0.0,0.0,0.0,0.0,test,0.000000,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10327,2020-12-16 22:11:00+00:00,1.0,1.0,0.0,0.0,0.0,test,0.419042,1,1,1,0,0,0
10328,2020-12-16 22:12:00+00:00,1.0,1.0,0.0,0.0,0.0,test,0.419042,1,1,1,0,0,0
10329,2020-12-16 22:13:00+00:00,1.0,1.0,0.0,0.0,0.0,test,0.419042,1,1,1,0,0,0
10330,2020-12-16 22:14:00+00:00,1.0,1.0,0.0,0.0,0.0,test,0.419042,1,1,1,0,0,0


In [13]:
#ideally we would need 0.306 for the threshold
accuracy_score(image_labels, master_copy_df["thresh2"])

0.7966166128112526

In [14]:
precision_recall_fscore_support(image_labels, master_copy_df["thresh2"], average = 'binary')

(0.8573309920983319, 0.7238695329873981, 0.7849678456591641, None)

In [15]:
#ideally we would need 0.306 for the threshold
accuracy_score(image_labels, master_copy_df["thresh3"])

0.7962364569473485

In [16]:
precision_recall_fscore_support(image_labels, master_copy_df["thresh3"], average = 'binary')

(0.8572056239015817, 0.7231282431430689, 0.7844792923200643, None)

In [17]:
accuracy_score(image_labels, master_copy_df["thresh4"])

0.789773807260977

In [18]:
precision_recall_fscore_support(image_labels, master_copy_df["thresh4"], average = 'binary')

(0.8566308243727598, 0.7086730911786508, 0.7756592292089249, None)

In [19]:
accuracy_score(image_labels, master_copy_df["thresh5"])

0.568142938604828

In [20]:
precision_recall_fscore_support(image_labels, master_copy_df["thresh5"], average = 'binary')

(0.9976635514018691, 0.15826538176426982, 0.27319257837492006, None)

In [21]:
accuracy_score(image_labels, master_copy_df["thresh6"])

0.5664322372172591

In [22]:
precision_recall_fscore_support(image_labels, master_copy_df["thresh6"], average = 'binary')

(1.0, 0.15455893254262418, 0.2677367576243981, None)

In [23]:
accuracy_score(image_labels, master_copy_df["thresh7"])

0.562630678578217

In [24]:
precision_recall_fscore_support(image_labels, master_copy_df["thresh7"], average = 'binary')

(1.0, 0.14714603409933283, 0.2565428109854604, None)

In [25]:
# precision_recall_fscore_support(image_labels, smokeynet_preds)

In [26]:
# precision_recall_fscore_support(image_labels, ensemble_preds)